In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
import requests

In [ ]:
model_name = "openai/clip-vit-base-patch32"

In [14]:
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print(f"Loaded {model_name} on {device}")

Loaded openai/clip-vit-base-patch32 on cuda


In [15]:
# Test  CLIP
def predict_similarity(image, texts):
    inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)
    
    return probs.cpu().numpy()

image_url = "https://images.unsplash.com/photo-1574158622682-e40e69881006"
image = Image.open(requests.get(image_url, stream=True).raw)

texts = ["a photo of a cat", "a photo of a dog"]
probabilities = predict_similarity(image, texts)
print(probabilities)

[[0.99633646 0.00366351]]
